In [4]:
import pandas as pd
import numpy as np

ages = np.array([20, 22,25,27,21,23,37,31,61,45,41,32]) #一些年龄数据
# 通过 pandas中的cut方法可以分割数据
# factory = pd.cut(ages,4)     #arr原始数据 , 4：要分割成几段
factory = pd.cut(ages,4,labels=['Youth', 'YoungAdult', 'MiddleAged', 'Senior']) #lable，对于每个类别可以自己命名
# factory = pd.cut(arr,bins=[18,25,35,60,100],labels=['a','b','c','d']) #bins 自己指定的分割界限
# factory.dtype  #CategoricalDtype，可以看到，cut后返回的是一个Categorical 类型的对象
test = np.array(factory)  #获取出分类后的数据
test

array(['Youth', 'Youth', 'Youth', 'Youth', 'Youth', 'Youth', 'YoungAdult',
       'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult'],
      dtype=object)

In [5]:
# 下面看下等量划分
# 通过 pandas中的qcut方法可以分割数据
factory = pd.qcut(ages,4) 
# factory
factory.value_counts()   #可以看到，通过等量划分，每个类别中的数据个数都一样

(19.999, 22.75]    3
(22.75, 29.0]      3
(29.0, 38.0]       3
(38.0, 61.0]       3
dtype: int64

### one-hot编码

对于类别型数据，最主要的一个处理，就是进行one-hot编码,看具体例子

In [6]:
testdata = pd.DataFrame({'age':[4,6,3,3],'pet':['cat','dog','dog','fish']})
testdata

,age,pet
0,4,cat
1,6,dog
2,3,dog
3,3,fish


In [8]:
#第一种方法，通过pandas中的提供的get_dummies方法
pd.get_dummies(testdata,columns=['pet']) #第一个参数为原始数据，columns传入需要编码转换的特征，可以为多个，返回新的数据

,age,pet_cat,pet_dog,pet_fish
0,4,1,0,0
1,6,0,1,0
2,3,0,1,0
3,3,0,0,1


In [9]:
testdata.pet.values.reshape(-1,1)

array([['cat'],
       ['dog'],
       ['dog'],
       ['fish']], dtype=object)

In [10]:
#第二种方法，使用sklearn中的OneHotEncoder方法

from sklearn.preprocessing import OneHotEncoder
OneHotEncoder().fit_transform(testdata.age.values.reshape(-1,1)).toarray() 

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [11]:
#OneHotEncoder不能对String型的数值做处理,对String类型做处理的话需要先进行转换
# OneHotEncoder().fit_transform(testdata.pet.values.reshape(-1,1)).toarray()  #会报错
from sklearn.preprocessing import LabelEncoder
petvalue = LabelEncoder().fit_transform(testdata.pet)
print(petvalue)  # [0 1 1 2] 将字符串类别转换成整型类别
OneHotEncoder().fit_transform(petvalue.reshape(-1,1)).toarray() #可以看到结果和上面通过get_dummies转换出的结果相同

[0 1 1 2]


/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### 特征降维

在实际项目中，并不是维数越高越好，为什么要进行降维操作，主要是出于以下考虑

1. 特征维数越高，模型越容易过拟合，此时更复杂的模型就不好用。
2. 相互独立的特征维数越高，在模型不变的情况下，在测试集上达到相同的效果表现所需要的训练样本的数目就越大。
3. 特征数量增加带来的训练、测试以及存储的开销都会增大。
4. 在某些模型中，例如基于距离计算的模型KMeans，KNN等模型，在进行距离计算时，维度过高会影响精度和性能。
5. 可视化分析的需要。在低维的情况下，例如二维，三维，我们可以把数据绘制出来，可视化地看到数据。当维度增高时，就难以绘制出来了。
6. 在机器学习中，有一个非常经典的维度灾难的概念。